In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 6.7MB/s 
     |████████████████████████████████| 61kB 23.2MB/s 
     |████████████████████████████████| 471kB 39.9MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536233 sha256=d49f05db9645046d104b32e1f79f57eb1b3a2b71fde79b144be28aa410ef4622
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=899ccac91a59852f1a7565b5e59b79ff75396d4882403efbf8d8d6f22720a270
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=bfd00367ba298226bac0de8371e48c523705d535916038515b9ac6a849f754a1
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule
import pycuda.autoinit

In [ ]:
module = SourceModule("""
__global__ void add_two_array_3d(int nx, int ny, int nz, float *res, float *arr1, float *arr2){
  int x = threadIdx.x + blockDim.x * blockIdx.x;
  int y = threadIdx.y + blockDim.y * blockIdx.y;
  int z = threadIdx.z + blockDim.z * blockIdx.z;
  int ijk = nx * ny * z + nx * y + x;
  if (x < nx && y < ny && z < nz){
      res[ijk] = arr1[ijk] + arr2[ijk];
  }
}
""")

In [ ]:
add_two_array = module.get_function("add_two_array_3d")

In [ ]:
num_x = np.int32(3)
num_y = np.int32(3)
num_z = np.int32(3)
num_components = num_x * num_y * num_z

In [ ]:
arr1 = np.arange(num_components, dtype=np.float32).reshape(num_z, num_y, num_x)

In [ ]:
arr2 = np.ones([num_z, num_y, num_x], dtype=np.float32)

In [ ]:
res_gpu = gpuarray.zeros([num_z, num_y, num_x], dtype=np.float32)

In [ ]:
arr1_gpu = gpuarray.to_gpu(arr1)
arr2_gpu = gpuarray.to_gpu(arr2)

In [ ]:
threads_per_block = (6, 6, 6)
block_x = math.ceil(num_x / threads_per_block[0])
block_y = math.ceil(num_y / threads_per_block[1])
block_z = math.ceil(num_z / threads_per_block[2])
blocks_per_grid = (block_x, block_y, block_z)

In [ ]:
add_two_array(num_x, num_y, num_z, res_gpu, arr1_gpu, arr2_gpu, block=threads_per_block, grid=blocks_per_grid)

In [ ]:
res_gpu.get()

array([[[ 1.,  2.,  3.],
        [ 4.,  5.,  6.],
        [ 7.,  8.,  9.]],

       [[10., 11., 12.],
        [13., 14., 15.],
        [16., 17., 18.]],

       [[19., 20., 21.],
        [22., 23., 24.],
        [25., 26., 27.]]], dtype=float32)